In [39]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import MinMaxScaler

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow import keras

from sklearn.metrics import confusion_matrix, classification_report

In [40]:
df = pd.read_csv("../input/telco-customer-churn/WA_Fn-UseC_-Telco-Customer-Churn.csv")
df.head()

In [41]:
df.drop('customerID',axis='columns',inplace=True)
df.dtypes

In [42]:
df = df[df['TotalCharges'] != " "]
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'])

In [43]:
for col in df.columns:
    print(f'{col} : {df[col].unique()}')

In [44]:
df.replace('No phone service', 'No', inplace=True)
df.replace('No internet service', 'No', inplace=True)


df.replace({'Male' : 1, 'Female' : 0},inplace=True)

yes_no_cols = ['Partner','Dependents','PhoneService','MultipleLines','OnlineSecurity',
               'OnlineBackup','DeviceProtection','TechSupport','StreamingTV',
               'StreamingMovies','PaperlessBilling','Churn']
for col in yes_no_cols:
    df[col].replace({'Yes' : 1, 'No' : 0},inplace=True)

df

In [45]:
for col in df.columns:
    print(f'{col} : {df[col].unique()}')

In [46]:
scaling_cols = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()
df[scaling_cols] = scaler.fit_transform(df[scaling_cols])
df

In [47]:
df = pd.get_dummies(data=df, columns=['InternetService','Contract','PaymentMethod'])
df.columns

In [48]:
df.drop(['PaymentMethod_Credit card (automatic)','Contract_Month-to-month','InternetService_No'],axis='columns',inplace=True)

In [49]:
df.head()

In [50]:
y = df['Churn']
X = df.drop('Churn',axis='columns')

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2)

In [51]:
X_train.shape

In [52]:
X_test.shape

In [53]:
model = keras.Sequential([
    keras.layers.Dense(18, input_shape=(23,),activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
             loss='binary_crossentropy',
             metrics=['accuracy'])

model.fit(X_train,y_train,epochs=50)

In [54]:
model.evaluate(X_test,y_test)

In [55]:
y_p = model.predict(X_test)
y_pred = []
for i in y_p:
    if i>0.5:
        y_pred.append(1)
    else:
        y_pred.append(0)

In [56]:
print(classification_report(y_test,y_pred))

In [57]:
cm = confusion_matrix(y_test,y_pred)
plt.figure(figsize=(10,12))
sns.heatmap(cm,annot=True,fmt='d')
plt.xlabel('Prediction')
plt.ylabel('Truth')